In [1]:
#convert the data frame to lib ffm format
def data_frame_to_lib_ffm(frame,key,field_list): #no le currenlty
    new_frame=pd.DataFrame()
    
    for column_name in frame.columns:
        if column in field_list:
            if column_name!=key:
                col=[str(field_list[column_name])+':'+str(row)+':1' for row in frame[column_name]]         
            else:
                col=frame[column_name]
            new_frame[column_name]=col
    return new_frame

In [3]:
import numpy as np 
import pandas as pd 
import time
from util import fast_mapk
from sklearn.preprocessing import LabelEncoder
import time
import pickle #load encoder

In [4]:
#get data

df_train = pd.read_csv( "clicks_train.csv") #100,000
df_test = pd.read_csv( "clicks_test.csv") #100,000
events = pd.read_csv( "events.csv") #100,000
content=pd.read_csv('promoted_content.csv')
docs_cats_frame=pd.read_csv('documents_categories.csv')
docs_cats_frame=docs_cats_frame.rename(columns={'document_id':'document_id_y'})

/home/ubuntu/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
content.head()

,ad_id,document_id,campaign_id,advertiser_id
0,1,6614,1,7
1,2,471467,2,7
2,3,7692,3,7
3,4,471471,2,7
4,5,471472,2,7


In [20]:
len(content.document_id.unique())

185709

In [21]:
df_train.head()

,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0
2,1,144739,1
3,1,156824,0
4,1,279295,0


In [25]:
joined_train_clicks.head(10)

,display_id,ad_id,clicked,uuid,document_id_x,timestamp,platform,geo_location,document_id_y,campaign_id,advertiser_id,category_id,confidence_level
0,1,42337,0,cb8c55702adb93,379743,61,3,US>SC>519,938164,5969,1499,2100,0.92
1,1,42337,0,cb8c55702adb93,379743,61,3,US>SC>519,938164,5969,1499,1408,0.07
2,320369,42337,0,aeed1d38f8e3e3,969916,29787173,1,US>IL>602,938164,5969,1499,2100,0.92
3,320369,42337,0,aeed1d38f8e3e3,969916,29787173,1,US>IL>602,938164,5969,1499,1408,0.07
4,350833,42337,1,7482d935c4ec22,1628975,31407245,3,US>NC>550,938164,5969,1499,2100,0.92
5,350833,42337,1,7482d935c4ec22,1628975,31407245,3,US>NC>550,938164,5969,1499,1408,0.07
6,367648,42337,1,84d858a3da5fc6,1628975,32253103,3,US>PA>566,938164,5969,1499,2100,0.92
7,367648,42337,1,84d858a3da5fc6,1628975,32253103,3,US>PA>566,938164,5969,1499,1408,0.07
8,393030,42337,0,440adb00eae522,1459603,33522620,1,US>NJ>501,938164,5969,1499,2100,0.92
9,393030,42337,0,440adb00eae522,1459603,33522620,1,US>NJ>501,938164,5969,1499,1408,0.07


In [14]:
min(content['document_id']),max(content['document_id'])

(1, 2999286)

In [5]:
#take fraction of the data:
t=time.time()
uniques=len(df_train.display_id.unique())
msk=np.random.rand(uniques)<0.3 #fraction
msk=df_train.display_id.unique()[msk]
train = df_train[df_train.display_id.isin([msk])]
print len(train)
docs_cats_frame1=docs_cats_frame.drop_duplicates(subset='document_id_y',keep='first')#remove duplicates
print time.time()-t

26137112
11.8732459545


In [6]:
#split train test by display id
uniques=len(train.display_id.unique())
t=time.time()
msk = np.random.rand(uniques) < 0.8
msk=train.display_id.unique()[msk]
print time.time()-t
t=time.time()
train = df_train[df_train.display_id.isin([msk])]
print time.time()-t
t=time.time()
val = df_train[~df_train.display_id.isin([msk])]
print time.time()-t
print len(train),len(val)

0.473602056503
7.77174019814
8.81528401375
20902930 66238801


In [8]:
#join page views and events - training run

t=time.time()
joined_train_clicks=train.merge(events)
joined_train_clicks=joined_train_clicks.merge(content,on='ad_id')
print time.time()-t
t=time.time()
joined_train_clicks=joined_train_clicks.merge(docs_cats_frame,on='document_id_y')

print time.time()-t
t=time.time()
joined_val_clicks=val.merge(events)
joined_val_clicks=joined_val_clicks.merge(content,on='ad_id')
joined_val_clicks=joined_val_clicks.merge(docs_cats_frame,on='document_id_y')
print time.time()-t




21.3521990776
7.27861499786
61.9763579369


In [9]:
joined_train_clicks=joined_train_clicks[~joined_train_clicks['uuid']\
                                        .isin(['5cda9845a1b9be', '7e8aa06b36db6a' ,'83e9ec48908c6a', 'f4bb634c3871b9','558ba104e8a37c'])]

In [10]:
#remove some uids because they are not in label encoder :0


joined_val_clicks=joined_val_clicks[~joined_val_clicks['uuid']\
                                        .isin(['5cda9845a1b9be', '7e8aa06b36db6a' ,'83e9ec48908c6a', 'f4bb634c3871b9','558ba104e8a37c'])]

In [11]:
#set features
ad_ids=joined_train_clicks['ad_id'].append(joined_val_clicks['ad_id'])
uuids=joined_train_clicks['uuid'].append(joined_val_clicks['uuid'])
dest_doc_id=content['document_id'].unique()
docs_cats=docs_cats_frame['category_id'].unique()

In [35]:
#precprocess: create label encoder for uuid and document id
from sklearn.preprocessing import LabelEncoder
'''
le={}

t=time.time()
le['uuid']=LabelEncoder()
le['uuid'].fit(uuids)
print time.time()-t
t=time.time()

le['ad_id']=LabelEncoder()
le['ad_id'].fit(ad_ids)
print time.time()-t
t=time.time()

le['document_id']=LabelEncoder()  #off
le['document_id'].fit(document_ids)
print time.time()-t
t=time.time()

le['geo_location']=LabelEncoder() #off
le['geo_location'].fit(geo_locations)

print time.time()-t
'''
t=time.time()
le['dest_doc_id']=LabelEncoder() #destinatiom document
le['dest_doc_id'].fit(dest_doc_id)
print time.time()-t

t=time.time()
le['docs_cats']=LabelEncoder() 
le['docs_cats'].fit(docs_cats)
print time.time()-t



import pickle #saved at /datadrive


4.0621778965
0.000227928161621


In [36]:

with open('le.pickle', 'wb') as f:
    pickle.dump(le, f, -1)

In [97]:
#convert feautres to int - do once for test run, then write
column_diffs=[]
t=time.time()
train_users=le['uuid'].transform(joined_train_clicks['uuid'])
print time.time()-t
t=time.time()
test_users=le['uuid'].transform(joined_test_clicks['uuid'])
column_diffs.append(max(np.append(train_users,test_users)))
print time.time()-t
t=time.time()

test_ads=le['ad_id'].transform(joined_test_clicks['ad_id'])+column_diffs[0]
train_ads=le['ad_id'].transform(joined_train_clicks['ad_id'])+column_diffs[0]
column_diffs.append(max(np.append(test_ads,train_ads)))
print time.time()-t
t=time.time()



623.844252825
244.84198308
13.5361759663


ValueError: y contains new labels: [   1456   20895   20967 ..., 2996109 2996871 2996938]

In [99]:
t=time.time()
train_dest_docs=le['dest_doc_id'].transform(joined_train_clicks['document_id_y'])+column_diffs[1]
test_dest_docs=le['dest_doc_id'].transform(joined_test_clicks['document_id_y'])+column_diffs[1]
column_diffs.append(max(np.append(train_dest_docs,test_dest_docs)))
print time.time()-t
t=time.time()

t=time.time()
train_categories=le['docs_cats'].transform(joined_train_clicks['category_id'])+column_diffs[2]
test_categories=le['docs_cats'].transform(joined_test_clicks['category_id'])+column_diffs[2]
column_diffs.append(max(np.append(train_dest_docs,test_dest_docs)))
print time.time()-t
t=time.time()

12.946434021
10.9347259998


In [100]:
full_train_features_dict={'train_users':train_users,'train_ads':train_ads,'train_dest_docs':train_dest_docs,'train_categories':train_categories}
full_test_features_dict={'test_users':test_users,'test_ads':test_ads,'test_dest_docs':test_dest_docs,'test_categories':test_categories}

In [28]:
joined_train_clicks['uuid'][:10]

0    cb8c55702adb93
1    cb8c55702adb93
2    aeed1d38f8e3e3
3    aeed1d38f8e3e3
4    7482d935c4ec22
5    7482d935c4ec22
6    84d858a3da5fc6
7    84d858a3da5fc6
8    440adb00eae522
9    440adb00eae522
Name: uuid, dtype: object

In [40]:
le['uuid'].transform(joined_train_clicks['uuid'][:10])

array([15471843, 15471843, 13110227, 13110227,  8288505,  8288505,
        9636615,  9636615,  4292094,  4292094])

In [42]:
le['ad_id'].transform(joined_train_clicks['ad_id'][12600:12700])+19794961

array([19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931290,
       19931290, 19931290, 19931290, 19931290, 19931290, 19931

In [38]:
max(full_test_features_dict['test_users']),max(full_train_features_dict['train_users'])

(19794961, 19794960)

In [38]:
#convert feautres to int
column_diffs=[]
t=time.time()
train_users=le['uuid'].transform(joined_train_clicks['uuid'])
test_users=le['uuid'].transform(joined_val_clicks['uuid'])
column_diffs.append(max(np.append(train_users,test_users)))
print time.time()-t
t=time.time()

test_ads=le['ad_id'].transform(joined_val_clicks['ad_id'])+column_diffs[0]
train_ads=le['ad_id'].transform(joined_train_clicks['ad_id'])+column_diffs[0]
column_diffs.append(max(np.append(test_ads,train_ads)))
print time.time()-t
t=time.time()

t=time.time()
train_dest_docs=le['dest_doc_id'].transform(joined_train_clicks['document_id_y'])+column_diffs[1]
test_dest_docs=le['dest_doc_id'].transform(joined_val_clicks['document_id_y'])+column_diffs[1]
column_diffs.append(max(np.append(train_dest_docs,test_dest_docs)))
print time.time()-t
t=time.time()

t=time.time()
train_categories=le['docs_cats'].transform(joined_train_clicks['category_id'])+column_diffs[2]
test_categories=le['docs_cats'].transform(joined_val_clicks['category_id'])+column_diffs[2]
column_diffs.append(max(np.append(train_dest_docs,test_dest_docs)))
print time.time()-t
t=time.time()

KeyboardInterrupt: 

In [101]:
#write the docs with pickle
with open('full_train_features_dict.pickle', 'wb') as f:
    pickle.dump(full_train_features_dict, f, -1)
    
with open('full_test_features_dict.pickle', 'wb') as f:
    pickle.dump(full_test_features_dict, f, -1)

32192851

In [12]:
#this will change the previous data flow... make it better

pkl_file = open('/datadrive/le.pickle', 'rb')
le=pickle.load(pkl_file)

pkl_file = open('/datadrive/full_train_features_dict.pickle', 'rb')
full_train_features_dict=pickle.load(pkl_file)

pkl_file = open('/datadrive/full_test_features_dict.pickle', 'rb')
full_test_features_dict=pickle.load(pkl_file)

In [67]:
len(df_test)

32225162

In [63]:
#generate final data frames
t=time.time()
final_frame=pd.DataFrame({'uuid':train_users,'ad_id':train_ads,'clicked':joined_train_clicks['clicked'].astype(int)\
                         ,'dest_doc_id':train_dest_docs,'categories':train_categories})
print time.time()-t
t=time.time()
final_val_frame=pd.DataFrame({'uuid':test_users,'ad_id':test_ads,'clicked':joined_val_clicks['clicked'].astype(int)\
                             ,'dest_doc_id':test_dest_docs,'categories':test_categories})
print time.time()-t

1.27139687538


ValueError: array length 63922585 does not match index length 103540364

In [ ]:
#convert to libffm
    #the slowest part - can make faster w/ apply?
t=time.time()
lib_train_frame=data_frame_to_lib_ffm(final_frame,'clicked',{'clicked':-1, 'uuid':0, 'ad_id':1,'dest_doc_id':2})
print time.time()-t
t=time.time()
lib_val_frame=data_frame_to_lib_ffm(final_val_frame,'clicked',{'clicked':-1, 'uuid':0, 'ad_id':1,'dest_doc_id':2})
print time.time()-t


In [ ]:
#write to csv
t=time.time()
lib_train_frame[['clicked','uuid','ad_id','dest_doc_id']].to_csv('train_frame.csv',sep=' ',header=None,index=False)
print time.time()-t
t=time.time()
lib_val_frame[['clicked','uuid','ad_id','dest_doc_id']].to_csv('val_frame.csv',sep=' ',header=None,index=False)
print time.time()-t

In [ ]:
#read output
output_list=pd.read_csv('/datadrive/output',header=None)

print len(output_list),len(lib_val_frame)

test_set=joined_val_clicks[['display_id','ad_id','clicked']]#
test_set['prediction']=output_list[0]
#join to test

# sort test set
#test_set=test_set.sort(['display_id','prediction'],ascending=[True,False])
test_set['prediction'].fillna(0)
test_set=test_set.rename(columns = {'prediction':'likelihood'})
test_set.head()

In [ ]:
#test_set['p_ctr']=joined_val_clicks['likelihood']
test_set['likelihood']=(joined_val_clicks['likelihood']+test_set['likelihood'])/2
test_set.head()

In [ ]:
#calc mapk on validation set
util.fast_mapk(test_set) #columns: display id, likelihood,clicked

#add feauture
#1 join to data
#2 convert to int
#3 convert to libffm